In [55]:
import pandas as pd
import numpy as np
import collections
from copy import copy
from collections import defaultdict

import sys
sys.path.append('/dados01/workspace/ufmg.f01dcc/utils')
import google_drive

In [56]:
minimum_size = 20
forms_folder = '1V0Od5iLbeJ7E2oNcqnP4g-eQo89QwscD'
items_names = ['mascara', 'gasolina', 'dipirona', 'pneu', 'locacao', 'veiculo']
clusters = pd.read_csv("../dados/output/cluster_prices_statistics_year.csv.zip", sep=';')
items = pd.read_csv("../dados/output/items_clusters_train.csv.zip", sep=';')

In [57]:
def get_valid_clusters(clusterDf):
    all_clusters = clusterDf.groupby(['cluster','dsc_unidade_medida'])['count'].sum().sort_values(ascending=False)
    print('Grupos distintos:', len(all_clusters))
    print('Total de itens:', all_clusters.sum())
    valid_clusters = all_clusters[all_clusters >= minimum_size]
    print('Grupos distintos 20 itens ou mais:', len(valid_clusters))
    return [all_clusters, valid_clusters];

In [61]:
def get_unid_medida_info(clusterDf):
    un_medida_clusters = clusterDf.groupby(['dsc_unidade_medida']).sum().sort_values(ascending=False)
    
    print('Unidades de medida distintas: ', len(un_medida_clusters))
    
    un_medida_clusters = dict(clusterDf.groupby(['dsc_unidade_medida']).sum().sort_values(ascending=False).head(5))
    
    for k,v in un_medida_clusters.items():
        un_medida_clusters[k] = round(v/sum(un_medida_clusters.values()), 4)
    
    return dict(sorted(un_medida_clusters.items(), key=lambda x: x[1], reverse=True))

In [67]:
def get_item_info(info_dict, item_name, clusters, valid_clusters, un_medidas):
    info_dict['item_name'] = item_name
    info_dict['distinct_groups_count'] = int(len(clusters))
    info_dict['valid_groups_count'] = int(len(valid_clusters))
    info_dict['total_items'] = int(clusters.sum())
    info_dict['distinct_units'] = int(len(un_medidas))

    info_dict['units'] = []
    for key,value in un_medidas.items():
        info_dict['units'].append({key:round(value/sum(un_medidas.values()),2)})
    
    info_dict['clusters'] = []

In [70]:
def get_item_clusters_info(info_dict, valid_clusters):
    for cluster_info in valid_clusters.index.values[:5]:
        print('-'*40)
        cluster_dict = {}
    
        cluster_name = cluster_info[0]
        cluster_un_medida = cluster_info[1]
        cluster_dict['name'] = cluster_name
        cluster_dict['unit'] = cluster_un_medida
    
        print(f'## {cluster_name} ##')
        print(f'Unidade de medida: {cluster_un_medida}')
    
        cluster_items = items[(items.cluster == cluster_name) & (items.dsc_unidade_medida == cluster_un_medida)]
        cluster_descriptions = cluster_items.groupby('description').size().sort_values(ascending=False)
        total_descriptions = cluster_descriptions.values.sum()
    
        print('Tamanho únicas: ', len(cluster_items))
        print('Descrições únicas: ', len(cluster_items['description'].unique()))
        print('Descrições mais frequentes:', list(cluster_descriptions.index.values)[:5])
    
        cluster_dict['size'] = int(len(cluster_items))
        cluster_dict['distinct_descriptions'] = int(len(cluster_items['description'].unique()))
        cluster_dict['total_descriptions'] = int(cluster_descriptions.values.sum())
        cluster_dict['descriptions'] = []
    
        for desc in cluster_descriptions.index.values[:5]:
            cluster_dict['descriptions'].append({desc:round(cluster_descriptions[desc]/total_descriptions,2)})

        cluster_tokens = defaultdict(int)
        for item in cluster_descriptions.index.values:
            for token in item.split(' '):
                cluster_tokens[token] += 1
        cluster_tokens = sorted(cluster_tokens.items(), key=lambda x: x[1], reverse=True)
        
        tokensLimit = max(5, min(int(len(cluster_tokens)*0.05), 10))
        print('Tokens distintos:' , len(cluster_tokens))
        print('Tokens selecionados:' , tokensLimit)
        print('Tokens mais frequentes:', list(cluster_tokens)[:tokensLimit])
    
        cluster_dict['tokens'] = dict(cluster_tokens[:tokensLimit])
        info_dict['clusters'].append(cluster_dict)

In [74]:
for name in items_names:
    print(f'## {name} ##')
    item_clusters_df = clusters[clusters.cluster.str.contains(name)]
    item_df = items[(items.outlier != 1) & (items.cluster.str.contains(name))]
    all_clusters, valid_clusters = get_valid_clusters(item_clusters_df)
    
    item_json_info = {}
    
    un_medidas = get_unid_medida_info(all_clusters)
    get_item_info(item_json_info, name, all_clusters, valid_clusters, un_medidas)
    get_item_clusters_info(item_json_info, all_clusters)
    
    google_drive.writeJsonFile(name, item_json_info, forms_folder)

## mascara ##
Grupos distintos: 349
Total de itens: 8007
Grupos distintos 20 itens ou mais: 87
Unidades de medida distintas:  64
----------------------------------------
## mascara_26 ##
Unidade de medida: caixa
Tamanho únicas:  1304
Descrições únicas:  351
Descrições mais frequentes: ['mascara descartavel', 'mascara cirurgico descartavel', 'mascara cirurgico', 'mascara descartavel com elastico', 'mascara descartavel c elastico']
Tokens distintos: 134
Tokens selecionados: 6
Tokens mais frequentes: [('mascara', 352), ('descartavel', 209), ('elastico', 193), ('50', 182), ('c', 169), ('com', 152)]
----------------------------------------
## mascara_26 ##
Unidade de medida: unidade
Tamanho únicas:  925
Descrições únicas:  312
Descrições mais frequentes: ['mascara descartavel', 'mascara a95', 'mascara', 'mascara n 95', 'mascara descartavel azul']
Tokens distintos: 214
Tokens selecionados: 10
Tokens mais frequentes: [('mascara', 312), ('descartavel', 96), ('com', 43), ('elastico', 41), ('c',

Permission Id: 11918420385910730359
File ID: 1YRl1pt32pWWv2kaubU8KqBg017vSYczQ
## pneu ##
Grupos distintos: 660
Total de itens: 43417
Grupos distintos 20 itens ou mais: 338
Unidades de medida distintas:  40
----------------------------------------
## pneu_46 ##
Unidade de medida: unidade
Tamanho únicas:  11916
Descrições únicas:  1997
Descrições mais frequentes: ['pneu 175 70 r14', 'pneu 215 75 r 17 5', 'pneu 215 75 17 5', 'pneu 205 70 r15', 'pneu 205 70 r 15']
Tokens distintos: 740
Tokens selecionados: 10
Tokens mais frequentes: [('pneu', 2000), ('r', 554), ('16', 456), ('x', 414), ('70', 404), ('75', 329), ('5', 322), ('15', 280), ('205', 247), ('radial', 235)]
----------------------------------------
## pneu_44 ##
Unidade de medida: unidade
Tamanho únicas:  1868
Descrições únicas:  286
Descrições mais frequentes: ['pneu 175 70 r 14', 'pneu 175 70 14', 'pneu 185 r 14', 'pneu 185 70 r 14', 'pneu 175 65 r 14']
Tokens distintos: 156
Tokens selecionados: 7
Tokens mais frequentes: [('pneu

Tamanho únicas:  168
Descrições únicas:  41
Descrições mais frequentes: ['veiculo', 'veiculo tipo pick up', 'veiculo pick up', 'veiculo usado', 'veiculo tipo van']
Tokens distintos: 58
Tokens selecionados: 5
Tokens mais frequentes: [('veiculo', 41), ('tipo', 11), ('van', 4), ('pick', 3), ('up', 3)]
----------------------------------------
## veiculo_11 ##
Unidade de medida: unidade
Tamanho únicas:  131
Descrições únicas:  121
Descrições mais frequentes: ['veiculo ambulancia fiat ducato diesel ano modelo 2005', 'veiculo usado minibus diesel minimo 17 lugar 103 cv turbinado intercalado ano 2003', 'veiculo fiat fiorino fire ambulancia ano 2006 placa cmn 5170 chassi 9 bd', 'veiculo mercedes benz 371 rs onibus ano 1989 placa gvi 9642', 'veiculo gm astra sedan elite automovel ano 2004 2005 placa abd 3001']
Tokens distintos: 473
Tokens selecionados: 10
Tokens mais frequentes: [('veiculo', 121), ('ano', 121), ('placa', 88), ('modelo', 56), ('cor', 54), ('fiat', 47), ('chassi', 43), ('branco', 

In [72]:
item_json_info

{'item_name': 'mascara',
 'distinct_groups_count': 349,
 'valid_groups_count': 87,
 'total_items': 8007,
 'distinct_units': 5,
 'units': [{'kit': 0.28},
  {'caixa': 0.23},
  {'unidade': 0.18},
  {'pacote': 0.16},
  {'peca': 0.15}],
 'clusters': [{'name': 'mascara_26',
   'unit': 'caixa',
   'size': 1304,
   'distinct_descriptions': 351,
   'total_descriptions': 1304,
   'descriptions': [{'mascara descartavel': 0.11},
    {'mascara cirurgico descartavel': 0.07},
    {'mascara cirurgico': 0.03},
    {'mascara descartavel com elastico': 0.02},
    {'mascara descartavel c elastico': 0.02}],
   'tokens': {'mascara': 352,
    'descartavel': 209,
    'elastico': 193,
    '50': 182,
    'c': 169,
    'com': 152}},
  {'name': 'mascara_26',
   'unit': 'unidade',
   'size': 925,
   'distinct_descriptions': 312,
   'total_descriptions': 925,
   'descriptions': [{'mascara descartavel': 0.09},
    {'mascara a95': 0.05},
    {'mascara': 0.04},
    {'mascara n 95': 0.04},
    {'mascara descartavel azu